In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('../data/final_dataset.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True) 

# Using close and sentiment score
data = df[['Close']].values

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)